# HIV Infections in New York City

This analyzes the patterns and disparities of HIV infections in New York City. The aim is to find out how race/ethnicity, age group, gender and neighborhood influence the rate of HIV AIDS contamination.

Importing all required libraries

In [8]:
import requests
import pandas as pd
import json
from pathlib import Path
import sqlite3
import re

Dataset reading and dataframe called df_Gender created

In [9]:
df_Gender = pd.read_csv('Dataset/HIV_AIDS_by_Gender_by_Neighborhood_and_Race_Ethnicity.csv')
df_Gender

,YEAR,Borough,Neighborhood (U.H.F),SEX,RACE/ETHNICITY,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION"
0,2010,NaN,Greenpoint,Male,Black,6,330.4,0,0,5,275.3
1,2011,NaN,Stapleton - St. George,Female,Native American,0,0,0,0,0,0
2,2010,NaN,Southeast Queens,Male,All,23,25.4,5,21.7,14,15.4
3,2012,NaN,Upper Westside,Female,Unknown,0,0,0,0,0,0
4,2013,NaN,Willowbrook,Male,Unknown,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8971,2021,Staten\nIsland,Willowbrook,Male,Asian/Pacific\nIslander,2,24.2,0,0,0,0
8972,2021,Staten\nIsland,Willowbrook,Male,Black,0,0,0,NaN,0,0
8973,2021,Staten\nIsland,Willowbrook,Male,Latino/Hispanic,1,18.2,0,0,0,0
8974,2021,Staten\nIsland,Willowbrook,Male,Other/Unknown,0,0,0,NaN,1,153


Dataset reading and dataframe called df_Age created

In [10]:
df_Age = pd.read_csv('Dataset/HIV_AIDS_by_Age_Group_by_Neighborhood_and_Race_Ethnicity.csv')
df_Age

,YEAR,NEIGHBORHOOD,RACE/ETHNICITY,AGE,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION",Borough
0,2013,High Bridge - Morrisania,Native American,30 - 39,0,0,0,0,0,0,NaN
1,2010,East Harlem,Unknown,40 - 49,0,NaN,0,NaN,0,NaN,NaN
2,2011,High Bridge - Morrisania,Unknown,40 - 49,0,NaN,0,NaN,0,NaN,NaN
3,2012,Greenwich Village - SoHo,Native American,50 - 59,0,0,0,0,0,0,NaN
4,2011,Long Island City - Astoria,Multiracial,20 - 29,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21031,2021,Willowbrook,Black,60+,0,0,0,NaN,0,0,Staten\nIsland
21032,2021,Willowbrook,Latino/Hispanic,60+,0,0,0,NaN,0,0,Staten\nIsland
21033,2021,Willowbrook,Other/Unknown,60+,0,0,0,NaN,NaN,NaN,Staten\nIsland
21034,2021,Willowbrook,White,60+,0,0,0,NaN,1,5.6,Staten\nIsland


Quick check on the data type we are working with and engaging in data cleaning

In [11]:
# Checking the number of rows & columns in Gender dataset
df_Gender.shape

(8976, 11)

In [12]:
# Checking the number of rows & columns in Age dataset
df_Age.shape

(21036, 11)

In [13]:
#Slicing the number of rows in Age to match Gender
df_Age = df_Age.head(8976).copy()
df_Age.shape

(8976, 11)

In [14]:
# Checking unique values in Age before joining the datasets
pd.unique(df_Age["NEIGHBORHOOD"])

array(['High Bridge - Morrisania', 'East Harlem',
       'Greenwich Village - SoHo', 'Long Island City - Astoria',
       'Upper Eastside', 'West Queens', 'Chelsea - Clinton',
       'Southeast Queens', 'Unknown', 'East New York',
       'East Flatbush - Flatbush', 'Greenpoint', 'Canarsie - Flatlands',
       'Port Richmond', 'Central Harlem - Morningside Heights',
       'Hunts Point - Mott Haven', 'Williamsburg - Bushwick', 'Jamaica',
       'Lower Manhattan', 'Pelham - Throgs Neck', 'Crotona - Tremont',
       'Fordham - Bronx Park', 'Ridgewood - Forest Hills',
       'Downtown - Heights - Park Slope', 'Kingsbridge - Riverdale',
       'Flushing - Clearview', 'Washington Heights - Inwood',
       'Stapleton - St. George', 'Willowbrook',
       'Union Square - Lower Eastside', 'Borough Park',
       'South Beach - Tottenville', 'Fresh Meadows',
       'Bayside - Little Neck', 'Sunset Park', 'Southwest Queens',
       'Upper Westside', 'Coney Island - Sheepshead Bay', 'Rockaway',
    